## XGBoost

Chegamos ao nosso último método de ensemble, o XGBoost (e**X**treme **G**radient **Boost**ing).

Este método nada mais é que um gradient boosting, mas com algumas importantes modificações que lhe conferem o título de "extreme"! Em particular, duas alterações merecem destaque:

- A adição de procedimentos de regularização (L1 e L2!), o que melhora consideravelmente sua capacidade de generalização;

- A utilização de derivadas de segunda ordem (Hessiano) para o procedimento de gradiente.

Para quem quiser se aventurar mais, sugiro algumas boas leituras:

- [Este](https://shirinsplayground.netlify.app/2018/11/ml_basics_gbm/), explica bem as particularidades do XGBoost, além de dar uma boa introdução ao gradient boosting (o código é em R, então pode ignorar essa parte hehe);

- [Este](https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb), introduz bem o método, enquanto enfativa suas particularidades, com alguns detalhes matemáticos;

- [Este](https://xgboost.readthedocs.io/en/latest/tutorials/model.html), da própria documentação da biblioteca, traz uma explicação legal, e com alguns detalhes matemáticos;

- [Este](https://towardsdatascience.com/https-medium-com-vishalmorde-xgboost-algorithm-long-she-may-rein-edd9f99be63d), com uma discussão mais alto-nível (sem tantos detalhes) sobre o XGBoost e os motivos de seu sucesso.

Basicamente temos um GradientBoost Extreme:
- Quanto ao paralelismo, a construção sequêncial de arvores é paralelisada.
    - Temos o loop externo que enumera os nós folha de uma árvore e o segundo loop interno que calcula os recursos, o truque é primeiro preparar várias linhas de execução para cada folha e cada uma delas é disparada no momento do cálculo da hipotese.
- Poda da árvore: Após a expansão da árvore, pode ocorrer que os subconjuntos de nós folhas sejam muito pequenos e com grande efeito de overfitting e esse ajuste pode ser minizado, já que temos o critério de max_depth.
- Otimização de hardware.
- Já regulariza por Lasso e Ridge.
- Tratamento de valores faltantes, já "aprende" qual o melhor valor para adotar
- Aplica quantil ponderado nos dados.
- Validação cruzada integrado.

<img src='https://cdn-images-1.medium.com/max/1000/1*U72CpSTnJ-XTjCisJqCqLg.jpeg'/>

Alguns hiperparâmetros: ( https://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters )
- learning_rate: taxa de aprendizado, valores menores de passo usado para evitar overfitting. O intervalo é [0,1].
- max_depth: determina quão profundamente cada árvore pode crescer durante qualquer rodada de reforço.
- subsample: porcentagem de amostras utilizadas por árvore. O baixo valor pode levar ao underfitting.
- colsample_bytree: porcentagem de recursos usados por árvore. Alto valor pode levar a overfitting.
- n_estimators: número de árvores que você deseja construir.
- objective: determina a função de perda a ser usada como reg:linear para problemas de regressão, reg:logistic para problemas de classificação com apenas decisão, binary:logistic para problemas de classificação com probabilidade.
- Regularização
    - gamma: controla se um determinado nó será dividido com base na redução esperada na perda após a divisão. Um valor mais alto leva a menos divisões. Suportado apenas para alunos baseados em árvore.
    - alpha: Regularização L1 nos pesos das folhas. Um valor grande leva a mais regularização.
    - lambda: Regularização L2 nos pesos das folhas e é mais suave do que a regularização L1.
- Validação Cruzada
    - num_boost_round: denota o número de árvores que você constrói (análogo a n_estimators)
    - metrics: informa as métricas de avaliação a serem observadas durante o CV
    - early_stopping_rounds: termina o treinamento do modelo antecipadamente se a métrica de hold-out ("rmse" em nosso caso) não melhorar em um determinado número de rodadas.

Infelizmente, o sklearn não tem o XGBoost implementado :(

Mas, felizmente, existe uma biblioteca que o implementou, de maneira totalmente integrada ao sklearn!!

A biblioteca é a [XGBoost](https://xgboost.readthedocs.io/en/latest/).

Para instalar a biblioteca, o de sempre: ( https://xgboost.readthedocs.io/en/latest/install.html#python )

`!pip install xgboost`

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('dados/german_credit_data.csv', index_col=0)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [3]:
df['Risk'] = df['Risk'].replace({'good': 1, 'bad': 0})

In [4]:
target = 'Risk'

feat_num = ['Age', 'Credit amount', 'Duration']

feat_cat_or = ['Job', 'Saving accounts', 'Checking account']
feat_cat_no = ['Sex', 'Housing', 'Purpose']

In [5]:
X = df.drop(columns=target)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, 
                                                    random_state=42, stratify=y)

Então ele faz tudo sozinho?

In [6]:
from xgboost import XGBClassifier

In [7]:
estimador = XGBClassifier(random_state=42, enable_categorical=True)

estimador.fit(X_train, y_train)

ValueError: Experimental support for categorical data is not implemented for current tree method yet.

Vamos trabalhar...

In [8]:
feat_cat_or_seq = {
    'Job': [0, 1, 2, 3],
    'Saving accounts': ['little', 'moderate', 'quite rich', 'rich'],
    'Checking account': ['little', 'moderate', 'rich']
}

In [9]:
# Pipeline customizado
# entra: feat_cat_or_set + X
# sai: X com os valores que representam a ordem

from sklearn.base import BaseEstimator, TransformerMixin

class PipelineArrumaValoresOrdenadamente(BaseEstimator, TransformerMixin):
    # PipelineArrumaValoresOrdenadamente() > criando um objeto (bolo de laranja do kuma)
    def __init__(self, feat_cat_or_seq):
        self.feat_cat_or_seq = feat_cat_or_seq
    
    def __cria_seq_replace(self, lista):
        return dict([(valor, i) for i, valor in enumerate(self.feat_cat_or_seq[lista])])
    
    def fit(self, X, parametros=None):
        return self
    
    def transform(self, df):
        df_temp = df.copy()
        for feature in feat_cat_or_seq.keys():
            valores_feature_ordenado = self.__cria_seq_replace(feature)
            df_temp[feature] = df_temp[feature].replace(valores_feature_ordenado)
        return df_temp

In [10]:
trans_customizado = PipelineArrumaValoresOrdenadamente(feat_cat_or_seq) # Chama o __init__

X_train_transformado = trans_customizado.transform(X_train)
X_train_transformado.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
675,26,female,3,rent,0.0,NaN,4530,30,radio/TV
703,41,male,2,own,1.0,1.0,2503,30,business
12,22,female,2,own,0.0,1.0,1567,12,radio/TV
845,35,male,2,own,NaN,1.0,3976,21,furniture/equipment
795,22,female,2,rent,1.0,NaN,2301,9,furniture/equipment


In [11]:
def cria_pipe_pre():
    pipe_pre_num = Pipeline([
        ('pre_inp_num', SimpleImputer(strategy='mean')),
        ('pre_sc_std', StandardScaler())
    ])

    pipe_pre_cat_or = Pipeline([
        ('pre_ord_cat_or', PipelineArrumaValoresOrdenadamente(feat_cat_or_seq)),
        ('pre_inp_cat_or', SimpleImputer(strategy='constant', fill_value=-1)),
        ('pre_sc_std', StandardScaler())
    ])

    pipe_pre_cat_no = Pipeline([
        ('pre_inp_cat_no', SimpleImputer(strategy='most_frequent')),
        ('pre_tra_one', OneHotEncoder(drop='first'))
    ])

    pipe_pre = ColumnTransformer([
        ('pipe_pre_num', pipe_pre_num, feat_num),
        ('pipe_pre_cat_or', pipe_pre_cat_or, feat_cat_or),
        ('pipe_pre_cat_no', pipe_pre_cat_no, feat_cat_no)
    ])
    
    return pipe_pre

In [12]:
pipe_pre = cria_pipe_pre()

pipe_pre.fit_transform(X_train)

array([[-0.8254793 ,  0.48538394,  0.75514869, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.49370456, -0.24657815,  0.75514869, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17726167, -0.58457347, -0.72674571, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.40575897, -0.82759789, -0.89140065, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17726167,  0.28424784, -0.56209078, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.28521488, -0.7040996 , -1.22071051, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
pipe_xgb = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_xgb', XGBClassifier(random_state=42, nthread=31))
])

pipe_xgb.fit(X_train, y_train)

y_pred = pipe_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.55      0.57        60
           1       0.81      0.84      0.82       140

    accuracy                           0.75       200
   macro avg       0.70      0.69      0.70       200
weighted avg       0.75      0.75      0.75       200



In [16]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [17]:
pipe_xgb = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_xgb', XGBClassifier(random_state=42, nthread=31))
])

espaco_hiper = {
    'est_xgb__learning_rate': [0.01, 0.1, 0.15, 0.2]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=espaco_hiper,
    scoring='f1',
    cv=cv,
    verbose=3,
    n_jobs=31
)

grid_xgb.fit(X_train, y_train)

y_pred = grid_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

grid_xgb.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 2/5] END ........est_xgb__learning_rate=0.2;, score=0.868 total time=  19.6s
[CV 1/5] END ........est_xgb__learning_rate=0.2;, score=0.833 total time=  19.6s
[CV 1/5] END .......est_xgb__learning_rate=0.15;, score=0.838 total time=  19.9s
[CV 2/5] END .......est_xgb__learning_rate=0.15;, score=0.882 total time=  20.0s
[CV 4/5] END .......est_xgb__learning_rate=0.15;, score=0.815 total time=  20.1s
[CV 5/5] END .......est_xgb__learning_rate=0.15;, score=0.820 total time=  20.2s
[CV 5/5] END ........est_xgb__learning_rate=0.1;, score=0.833 total time=  20.3s
[CV 3/5] END .......est_xgb__learning_rate=0.15;, score=0.800 total time=  20.3s
[CV 1/5] END ........est_xgb__learning_rate=0.1;, score=0.846 total time=  20.6s
[CV 3/5] END ........est_xgb__learning_rate=0.1;, score=0.795 total time=  20.7s
[CV 2/5] END ........est_xgb__learning_rate=0.1;, score=0.867 total time=  20.9s
[CV 4/5] END ........est_xgb__learning_rate=0.1;,

{'est_xgb__learning_rate': 0.15}

#### https://xgboost.readthedocs.io/en/latest/parameter.html

In [18]:
pipe_xgb = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_xgb', XGBClassifier(random_state=42, nthread=31))
])

espaco_hiper = {
    'est_xgb__learning_rate': [0.01, 0.1, 0.15, 0.2],
    'est_xgb__alpha': [0.01, 0.3, 0.8],
    'est_xgb__colsample_bytree': [0.1, 0.4, 0.8, 1],
    'est_xgb__n_estimators': [50],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=espaco_hiper,
    scoring='f1',
    cv=cv,
    verbose=3,
    n_jobs=31
)

grid_xgb.fit(X_train, y_train)

y_pred = grid_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

grid_xgb.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 3/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.01, est_xgb__n_estimators=50;, score=0.824 total time=   4.7s
[CV 5/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.01, est_xgb__n_estimators=50;, score=0.824 total time=   5.1s
[CV 3/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.1, est_xgb__n_estimators=50;, score=0.824 total time=   5.3s
[CV 5/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.15, est_xgb__n_estimators=50;, score=0.835 total time=   5.4s
[CV 2/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.1, est_xgb__n_estimators=50;, score=0.833 total time=   5.5s
[CV 3/5] END est_xgb__alpha=0.01, est_xgb__colsample_bytree=0.1, est_xgb__learning_rate=0.15, est_xgb__n_estimators=50;, score=0.826 total time=   5.5s
[CV 4/5] END est_xgb__alpha=

{'est_xgb__alpha': 0.01,
 'est_xgb__colsample_bytree': 0.8,
 'est_xgb__learning_rate': 0.01,
 'est_xgb__n_estimators': 50}

In [12]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [7, 8], [9, 10], [11, 12], [13, 14], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0])
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TRAIN: [ 0  1  2  4  6  7  8  9 10 11] TEST: [3 5]
TRAIN: [ 1  2  3  4  5  6  7  9 10 11] TEST: [0 8]
TRAIN: [ 0  1  2  3  5  6  8  9 10 11] TEST: [4 7]
TRAIN: [ 0  1  2  3  4  5  7  8  9 10] TEST: [ 6 11]
TRAIN: [ 0  2  3  4  5  6  7  8  9 11] TEST: [ 1 10]
TRAIN: [ 0  1  3  4  5  6  7  8 10 11] TEST: [2 9]


/Users/gustavoperbone/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=6.
  warnings.warn(


- Boosting e estratégias evolucionárias na tarefa de regressao para a mineraçao de dados temporais
    - https://acervodigital.ufpr.br/handle/1884/3770?show=full
- XGBoosting visão geral
    - https://www.kdnuggets.com/2019/05/xgboost-algorithm.html